In [1]:
from svm_layer import svm_layer
import tensorflow as tf
from tqdm import tqdm
from pathlib import Path
import numpy as np
from PIL import Image
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import NASNetLarge
import os

In [ ]:
datagen = ImageDataGenerator(
    rescale=1.0/255,
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    vertical_flip=True,
    fill_mode='nearest',
    validation_split = 0.2
)
test_datagen = ImageDataGenerator(rescale=1.0/255)
notebook_dir = os.getcwd()
project_root = os.path.dirname(notebook_dir)

In [ ]:
train_dir = os.path.join(project_root,'input', 'waste-classification-data', 'DATASET', 'TRAIN')
test_dir = os.path.join(project_root,'input', 'waste-classification-data', 'DATASET', 'TEST')

In [ ]:
train_generator = datagen.flow_from_directory(
    train_dir,
    target_size=(331, 331),
    batch_size=32,
    class_mode='binary',
    shuffle=True,
    subset = 'training'
)
valid_generator = datagen.flow_from_directory(
    train_dir,
    target_size=(331, 331),
    batch_size=32,
    class_mode='binary',
    shuffle=True,
    subset = 'validation'
)
test_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size=(331, 331),
    class_mode='binary',
    shuffle=False
)

In [ ]:
nasnet_model = NASNetLarge(weights='imagenet', include_top=True)

In [ ]:
x = nasnet_model.layers[-2].output
svm = svm_layer(penalty_para=1.0)(x)

In [ ]:
model = tf.keras.Model(inputs = nasnet_model.input, outputs = svm)
model.summary()

In [ ]:
for l in model.layers[:-5]:
    l.trainable = False
model.compile(optimizer = 'adam', loss='hinge')

In [ ]:
model.fit(train_generator,
          epochs=10,
          validation_data=valid_generator,
          callbacks = [EarlyStopping(patience=5, monitor='val_loss', min_delta=0.01)]
)

In [ ]:
model.save('nasnet_svm.h5')